# Day 05: If You Give A Seed A Fertilizer

## Setup

In [1]:
import sys

data = [line for line in open("./inputs/day05.txt", "r").read().splitlines()]

## Part 1
First, get all the seeds (starting number), then iterate through all the maps and save the lowest mapped number. Name of the map is saved to ensure that map can't occur more than once.

In [2]:
seeds = [int(seed) for seed in data[0].split(': ')[1].split(" ")]

min_result = sys.maxsize

for seed in seeds:
    map_name = ""
    performed_maps = set()

    for i in range(2, len(data)):
        if len(data[i]) == 0:
            continue

        if not data[i][0].isnumeric():
            map_name = data[i]
            continue

        if map_name in performed_maps:
            continue

        destination, source, range_length = [int(n) for n in data[i].split(" ")]

        if seed in range(source, source + range_length):
            performed_maps.add(map_name)
            seed += (destination - source)

    if seed < min_result:
        min_result = seed

print(min_result)

177942185


## Part 2
Let's flip this shit around

In [3]:
seeds = []
mappings = []

map_index = -1
for i in range(len(data)):
    if i == 0:
        seed_input = [int(seed) for seed in data[0].split(': ')[1].split(" ")]
        for s, seed in enumerate(seed_input):
            if s % 2 == 0:
                seeds.append([seed, seed + seed_input[s + 1] - 1])
        continue

    if len(data[i]) == 0:
        continue

    if not data[i][0].isnumeric():
        map_index += 1
        continue

    destination, source, range_length = [int(n) for n in data[i].split(" ")]

    if len(mappings) < map_index + 1:
        mappings.insert(map_index, [[source, source + range_length - 1, destination - source]])
    else:
        mappings[map_index].append([source, source + range_length - 1, destination - source])


def split_range(seed_range, mapping_ranges):
    seed_start, seed_end = seed_range
    # print("seed", seed_start, seed_end, mapping_ranges)

    mapped_ranges = []
    for mapping_start, mapping_end, offset in mapping_ranges:

        # mapping covers completely
        if mapping_start <= seed_start and mapping_end >= seed_end:
            mapped_ranges.append([seed_start, seed_end, offset])
        # seed touches start of mapping
        elif seed_start <= mapping_start <= seed_end:
            # mapping covers end of seed
            if mapping_end >= seed_end:
                mapped_ranges.append([mapping_start, seed_end, offset])
            else:
                mapped_ranges.append([mapping_start, mapping_end, offset])
        # seed touches end of mapping
        elif mapping_start < seed_start <= mapping_end:
            mapped_ranges.append([seed_start, mapping_end, offset])

    unmapped_start = seed_start
    unmapped_ranges = []
    for mapped_start, mapped_end, _ in sorted(mapped_ranges, key=lambda x: x[0]):
        if unmapped_start < mapped_start:
            unmapped_ranges.append([unmapped_start, mapped_start - 1])
            unmapped_start = mapped_start
        elif seed_end > mapped_end:
            unmapped_ranges.append([mapped_end + 1, seed_end])
            unmapped_start = seed_end

    if not mapped_ranges:
        unmapped_ranges.append([seed_start, seed_end])

    # print("returning", [*[[start + off, end + off] for start, end, off in mapped_ranges], *unmapped_ranges])
    return [*[[start + off, end + off] for start, end, off in mapped_ranges], *unmapped_ranges]


min_result = sys.maxsize


for seed in seeds:
    queue = [[seed]]

    for map_id, map_ranges in enumerate(mappings):
        while len(queue[map_id]) > 0:
            popped_seed = queue[map_id].pop()
            split_ranges = split_range(popped_seed, map_ranges)
            
            if len(queue) <= map_id + 1:
                queue.insert(map_id+1, list(split_ranges))
            else:
                queue[map_id+1].extend(split_ranges)
            

    min_seed = min([seed[0] for seed in queue[7]])
    if min_seed < min_result:
        min_result = min_seed

print(min_result)

7921916
